In [1]:
from torch_geometric.data import Data
from DataBuilder import DataBuilder
from constants import TRIPLES_PATH, ENTITIES_PATH, RELATIONS_PATH
from LLMEncoder import LLMEncoder

/Users/I302238/.local/share/virtualenvs/psmg-imkg-gnn-qa-b42qFiTl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import RobertaTokenizer, RobertaModel
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaModel.from_pretrained("roberta-base")
llm_encoder = LLMEncoder(tokenizer, model)
data_builder = DataBuilder(TRIPLES_PATH, ENTITIES_PATH, RELATIONS_PATH, llm_encoder=llm_encoder)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
x = data_builder.get_x()
edge_index = data_builder.get_edge_index()

ValueError: Input nan is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.

In [ ]:
data = Data(x=data_builder.x, edge_index=data_builder.edge_index)

In [36]:
import pandas as pd
entities_data =pd.read_csv(ENTITIES_PATH)

In [29]:
entities_data[entities_data.iloc[:,1].isna()].iloc[:,1] = entities_data[entities_data.iloc[:,1].isna()].iloc[:,0].apply(lambda x: x.split('/')[-1])

In [39]:
entities_data[entities_data.iloc[:,1].isna()].iloc

,uri,label
6599,http://www.signavio.com/opal/SAP/CPM/BPX/27,NaN
8498,http://www.signavio.com/opal/SAP/CPM/BPX/NA,NaN
10219,http://www.signavio.com/opal/SAP/EARL/BAD/1.0,NaN
10220,http://www.signavio.com/opal/SAP/EARL/BAD/2.0,NaN
10221,http://www.signavio.com/opal/SAP/EARL/BAD/3.0,NaN
10591,http://www.signavio.com/opal/SAP/EARL/SAD/2208,NaN
23775,http://www.signavio.com/opal/SAP/SIG/BMM/2022,NaN
28894,http://www.signavio.com/opal/core/1.0,NaN
28895,http://www.signavio.com/opal/core/1.0.0,NaN
28896,http://www.signavio.com/opal/core/1.63.2,NaN


In [40]:
mask = entities_data.iloc[:,1].isna()
entities_data.loc[mask, "label"] = entities_data[entities_data.iloc[:,1].isna()].iloc[:,0].apply(lambda x: x.split('/')[-1])

In [41]:
entities_data[entities_data.iloc[:,1].isna()]

,uri,label
